As humans, we are able to recognize a person in two pictures taken under different conditions. Unfortunately this is not the case for the computer. 

The latter needs a way to tell if two images look similar or are different.
In general, a computer sees an image as a highly dimensional vector. Often, features are extracted from this vector and the image is represented in a lower dimension feature space.


The goal of this challenge was to determine a metric that was suitable to compare data points in a given space of features. Given two points represented in that feature space, $x,y \in F$, the assumption behind metric learning is that *F* is not Euclidean, so that the similarity degree between ***x*** and ***y*** may not be computed by the classical Euclidean distance.

I am going to present the different approaches I took in my quest to find a good metric.

# General formulation 

I have a set *D*, of 300000 images expressed in a feature space *F*.
$D = \{x_{1},x_{2},\dots,x_{n}\}$, where $x_{i} \in F$, where $F\subseteq \mathbb{R}^{m}$, with $m = 1500$. 

The main goal is to find a metric that has distance or at least metric properties for our images : $d(x_{i},x_{j})$. 

For every image, we also have a label. We are in a special case where we have a lot of classes with very few examples per classes.

We formulate the problem as pairwise constraints : 

Given two images $x_{i}$ and $x_{j}$, the pair $(x_{i},x_{i})$ has the following label : 
$$
\begin{align}
        y_{i,j} = \begin{cases}
                        +1 \text{ if $x_{i}$ and $x_{j}$ have the same label} \\
                        -1 \text{ else}
                    \end{cases}
\end{align}
$$

By formulating the problem with pairwise constraint, we have a supervised learning framework to our disposition. 

In the rest of this article, I will show the different approaches I took to determine the best metric : *d*. 

# Performance assessment
We assess the performance of our metric using a ROC curve. By varying the threshold, we have a different proportion of true positives and false positives. In our problem, we define performance as a special point of the ROC curve : The false negative ratio at a 0.1% of true positives. 

# Different approaches

For this challenge, I took the following approaches in chronological order : 

- Simple approach : Trying out different similarities
- Supervised method to determine a linear metric : Mahalanobis Distance
- Supervised method : Similarity neural network

In the rest of this article, I will detail the mentionned approaches. 

# Simple approach

In this challenge, we did not have information about our features space. The first metric I tried was the Euclidean distance between pairs of image.

This approach was very simple to implement and landed me a score of approximately **0.56**. 

**Still**, the script was very slow as I had to manipulate high dimensional vectors and matrices.
I was able to boost the results by normalizing the data. 
Also, I used the fact that we did not know a lot about the feature space to perform some dimensionnality reduction on the data. I applied a PCA and proceeded empirically to find a number of dimension that captured variance without loosing to much data. I obtained good results by keeping n = 1500 / 2 dimensions.

After trying the euclidean distance, I explored different distances and similarities. Fortunately, it was very quick to iterate on the different similarities and distances and I was able to manually test them. 

Among the distances and similarities that performed the best were : 
- **Bray-curtis distance**
- Canberra distance
- Chebyshev distance
- Cosine distance. 

I obtained the best score with the Braycurtis distance : **0.213249303825**.


# Mahalanobis distance for linear metric learning

Given the fact that we had at our disposition a good supervised learning framework, I tried to harness this framework with pairwise constraints to adapt a similarity function which is the Mahalanobis distance. This distance is parametrized by a matrix $A \in \mathbb{R}^{m,m}$, such that : 
$$
\begin{align}
    d(x,y) = (x-y)^{T} = (x-y)^{T} A (x-y) = || x - y || ^{2}_{A}
\end{align}
$$

This function verifies the properties of symmetry, non-negativity (A is psd), and satisifies triangular inequality. 

This metric was learnt by minimizing a loss function on the pairs constraints. The loss function used was the hinge-loss function. 

Given the high dimensionnality of the matrix, I could not use a classic gradient descent, but a stochastic gradient descent performed well. I think that I could get a better convergence by implementing a minibatch version of the stochastic gradient descent.

Here is the following steps I used to get my best submission with the mahalanobis distance : 

- Load data and generate pairs to create a training set and a validation set
- Normalize the data 
- Reduce dimension to 750 dimensions with a PCA
- Set learning rate to 0.008, and with 10000 iterations

I think that I got a score of approximately : **0.33**. 

This parametrization is simple but it allows the distance to model **only linear and relationships among features**. 

# Similarity Neural Network

I wanted to take into account possible non linear relationships among features and decided that I would use my supervised framework to train a neural network to act as a similarity metric. The similarity Neural Network consists of a feedforward multi-layer perceptron trained to learn the similarity for pairs of images. 

The input vector of the neural net is the concatenation of both images in their features space : 
$[x_{i},x_{j}] \in \mathbb{R}^{2m}$. The output of this similarity neural network is $f(x_{i},x_{j},\theta)$, which is a real number between 0 and 1. $\theta$ represents the parameters of the neural network. 

Formally, this neural network can be seen as similarity has it verifies the properties of non negativity and simmetry. 

I followed a recurring structure from the litterature for this neural network. The neural network had the following architecture : 

<img src="neural.png"></img>

I used a simple implementation of a Multi Layer Perceptron adjusting the correct number of input units (2 * 1500), output units (1) with a single hidden layer with 100 units. I used the following value retrieved from the last layer of the neural network **as my metric** : 
$$
\begin{align}
    d(x_{i},x_{j},\theta) = \mathbb{P}(Y = 1 \mid (x_{i},x_{j},\theta))
\end{align}

This computation was very heavy and I had to write GPU optimized code to train my neural network. I set up an Amazon AWS spot request instance and chose the g2.xlarge instance for GPU calculations. 
Using the Theano library I was able to create a MLP that is trained using GPUs. 

**UNFORTUNATELY**, I did not have enough time to train my neural network. I heard that tuning a neural network was hard and it was the opportunity for me to verify it ... I also lost a lot of hours trying to get my head around Theano. I was not able to make a submission with my neural network ... 